In [18]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gensim
import nltk
from scipy.spatial import distance
import re
from random import shuffle
import string
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn import utils
from sklearn.model_selection import train_test_split
import ast
import pickle
import pprint
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Model


In [19]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [20]:
df_final = pd.read_csv('dataset/movies.csv', index_col=0) 
df_final.dropna(inplace=True)
df_final.reset_index(drop=True, inplace=True)
df_final['Description'] = df_final['Description'].apply(lambda x: str(x).strip())

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("turkish"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    new_string = text.translate(str.maketrans('', '', string.punctuation))
    return new_string

# Description only
df_final['cleaned'] = df_final['Description']
# Stars + Directors + Description
df_final = df_final.assign(combine=[(f'''{str(df_final["Title"][i]).strip("[]").replace("'", "").replace(",",".")} ,
{str(df_final["Genres"][i]).strip("[]").replace("'", "").replace(",",".")} ,
{str(df_final["Stars"][i]).strip("[]").replace("'", "").replace(",",".")} ,
{str(df_final["Stars"][i]).strip("[]").replace("'", "").replace(",",".")} ,
{str(df_final["Director"][i]).strip("[]").replace("'", "").replace(",",".")},
{str(df_final["Genres"][i]).strip("[]").replace("'", "").replace(",",".")},
{str(df_final["Genres"][i]).strip("[]").replace("'", "").replace(",",".")}''').lower() for i in range(len(df_final["Title"]))])
# Title + Description
df_final = df_final.assign(temp_f=[(f'''{str(df_final["Stars"][i]).strip("[]").replace("'", "").replace(",",".")}, {str(df_final["Director"][i]).strip("[]").replace("'", "").replace(",",".")}, {df_final['Description']}''').lower() for i in range(len(df_final["Title"]))])

df_final['cleaned'] = df_final.cleaned.apply(func = make_lower_case)
df_final['cleaned'] = df_final.cleaned.apply(func = remove_stop_words)
df_final['cleaned'] = df_final.cleaned.apply(func = remove_punctuation)
df_final['cleaned'] = df_final.cleaned.apply(func = remove_html)

df_final['combine'] = df_final['combine'].apply(func = make_lower_case)
df_final['combine'] = df_final['combine'].apply(func = remove_stop_words)
df_final['combine'] = df_final['combine'].apply(func = remove_punctuation)



#### Model 1

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(row['combine']), tags=[row['Title']]) for index, row in df_final.iterrows()]



In [22]:
print(tagged_data[471].tags, ': ', (tagged_data[471].words))


['Yüzüklerin Efendisi: İki Kule'] :  ['yüzüklerin', 'efendisi', 'i̇ki', 'kule', 'fantastik', 'macera', 'elijah', 'wood', 'sean', 'astin', 'viggo', 'mortensen', 'elijah', 'wood', 'sean', 'astin', 'viggo', 'mortensen', 'peter', 'jackson', 'fantastik', 'macera', 'fantastik', 'macera']


In [23]:
model_dbow = Doc2Vec(
    dm=0, dbow_words=1,
    vector_size=200, window=8, epochs=20, workers=12,
)

model_dbow.build_vocab(tagged_data)
print(model_dbow)
    


2023-02-22 18:30:25,498 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow+w,d200,n5,w8,mc5,s0.001,t12>', 'datetime': '2023-02-22T18:30:25.498286', 'gensim': '4.3.0', 'python': '3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-02-22 18:30:25,499 : INFO : collecting all words and their counts
2023-02-22 18:30:25,499 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-02-22 18:30:25,537 : INFO : PROGRESS: at example #10000, processed 225567 words (5892077 words/s), 26085 word types, 9472 tags
2023-02-22 18:30:25,621 : INFO : collected 42687 word types and 17513 unique tags from a corpus of 18492 examples and 414804 words
2023-02-22 18:30:25,622 : INFO : Creating a fresh vocabulary
2023-02-22 18:30:25,650 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 10246 unique words (24.00% of original 42687, drops 32441)', 'datetime

Doc2Vec<dbow+w,d200,n5,w8,mc5,s0.001,t12>


In [24]:
model_dbow.train(tagged_data, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)
model_dbow.save("doc2vec.model")


2023-02-22 18:30:25,803 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 12 workers on 10246 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=8 shrink_windows=True', 'datetime': '2023-02-22T18:30:25.803501', 'gensim': '4.3.0', 'python': '3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-02-22 18:30:26,988 : INFO : EPOCH 0 - PROGRESS: at 59.90% examples, 156785 words/s, in_qsize 17, out_qsize 0
2023-02-22 18:30:27,128 : INFO : EPOCH 0: training on 414804 raw words (297418 effective words) took 1.3s, 226331 effective words/s
2023-02-22 18:30:28,337 : INFO : EPOCH 1 - PROGRESS: at 59.90% examples, 153212 words/s, in_qsize 17, out_qsize 0
2023-02-22 18:30:28,485 : INFO : EPOCH 1: training on 414804 raw words (297129 effective words) took 1.3s, 220399 effective words/s
2023-02-22 18:30:29,735 : INFO : EPOCH 2 - PROGRESS: at 59.90% examples, 148354 words/s,

In [25]:
def search_by_tag(tag):
    for doc in tagged_data:
        if doc.tags[0] == tag:
            return " ".join(doc.words)

def return_most_similar(mov_title) -> list:
  tokens = remove_html(remove_punctuation(remove_stop_words(make_lower_case(search_by_tag(mov_title))))).split()
  vector = model_dbow.infer_vector(tokens)
  most_similar = model_dbow.dv.most_similar([vector], topn=20) 
  return most_similar

def return_token(mov_title) -> list:
  tokens = remove_html(remove_punctuation(remove_stop_words(make_lower_case(search_by_tag(mov_title))))).split()
  return tokens

def return_most_similar_token(vec) -> list:
  # len(df_final['Title']
  most_similar = model_dbow.dv.most_similar([vec],topn=20)
  return most_similar
pp = pprint.PrettyPrinter(indent=4)
seed_text = 'Iron Man'
x = return_most_similar(seed_text)
pp.pprint(x)
print(search_by_tag(seed_text))

[   ('Iron Man', 0.9398001432418823),
    ('Iron Man 3', 0.8286119699478149),
    ('Iron Man 2', 0.8158987760543823),
    ('Yenilmezler', 0.7131946086883545),
    ('Red Tails', 0.7069835066795349),
    ('Sky Captain ve Yarının Dünyası', 0.6855823993682861),
    ('Avengers: Endgame', 0.6840946674346924),
    ('Yenilmezler: Ultron Çağı', 0.6807721853256226),
    ('Avengers: Sonsuzluk Savaşı', 0.6733619570732117),
    ('Jurassic World: Hakimiyet', 0.6591430306434631),
    ('Air America', 0.6523217558860779),
    ('Sherlock Holmes', 0.6520013213157654),
    ('Örümcek-Adam: Eve Dönüş', 0.6508820652961731),
    ('Yüksek Gerilim', 0.6480500102043152),
    ('Gothika', 0.6461778879165649),
    ('Dövüş', 0.645106315612793),
    ('Fur: An Imaginary Portrait of Diane Arbus', 0.6416605710983276),
    ('A Scanner Darkly', 0.639180600643158),
    ('Tuff Turf', 0.638505220413208),
    ('Kanıt', 0.6371035575866699)]
iron man aksiyon bilimkurgu robert downey jr terrence howard gwyneth paltrow robert dow

In [26]:
seed_text = 'Dune:Çöl Gezegeni'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Dune:Çöl Gezegeni', 0.9543393850326538),
    ('Dune: Part Two', 0.9015074372291565),
    ('Hayat', 0.718077540397644),
    ('Ex Machina', 0.6980426907539368),
    ('Metal Gear Solid', 0.6805034279823303),
    ('Trendeki Kız', 0.6732518076896667),
    ('Star Wars: Son Jedi', 0.6704469323158264),
    ('Kardan Adam', 0.6681733727455139),
    ("Star Wars: Skywalker'ın Yükselişi", 0.6627540588378906),
    ("Won't Back Down", 0.6611697673797607),
    ('Mojave', 0.6510500311851501),
    ('Her Şey Olacağına Varır', 0.649515688419342),
    ('Zihin Gezgini', 0.647402822971344),
    ('Zehirli Element', 0.6454704403877258),
    ('Doktor Uyku', 0.638755202293396),
    ('Francis And The Godfather', 0.6355670094490051),
    ('London', 0.6329144239425659),
    ('Beni Adınla Çağır', 0.6269508004188538),
    ('Teen Spirit', 0.6261535882949829),
    ('Ocak Ayının İki Yüzü', 0.6254170536994934)]


In [27]:
seed_text = 'Yüzüklerin Efendisi: İki Kule'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Yüzüklerin Efendisi: İki Kule', 0.9456788301467896),
    ('Yüzüklerin Efendisi: Yüzük Kardeşliği', 0.8622255921363831),
    ('On Üç Yaşam', 0.6091465950012207),
    ('Kurbağa Prens', 0.6070922613143921),
    ('Woodlawn', 0.5981312990188599),
    ('The Goonies', 0.5964442491531372),
    ('Şiddetin Tarihçesi', 0.5905812382698059),
    ('Rumpelstiltskin', 0.5901864171028137),
    ('Tuhaf Bir Sihir', 0.5881508588790894),
    ('Try Seventeen', 0.5879155993461609),
    ('Yeşil Rehber', 0.5850781798362732),
    ('Yeşil Sokak Holiganları', 0.5835225582122803),
    ('Hero Mode', 0.5830981135368347),
    ('Time Bandits', 0.5801665186882019),
    ('Kanun Benim', 0.579239010810852),
    ('Düşüş', 0.5744485259056091),
    ('Offence, The', 0.5720624327659607),
    ('Şark Vaatleri', 0.568449854850769),
    ('Gerçeğin İki Yüzü', 0.5681962966918945),
    ('Pitch Perfect', 0.5618053674697876)]


In [28]:
seed_text = 'Rashômon'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Rashômon', 0.9667608141899109),
    ('Yüksek ve Alçak', 0.886763334274292),
    ('Yojimbo', 0.8776783347129822),
    ('Sanjuro', 0.8773446679115295),
    ('Kanlı Taht', 0.8740440607070923),
    ('Ran', 0.8683363795280457),
    ('Akahige', 0.8680164217948914),
    ('Dava Vekili', 0.8570104837417603),
    ('Yedi Samuray', 0.8538091778755188),
    ('Akasen Chitai', 0.8534561991691589),
    ('Floating Weeds', 0.8412061333656311),
    ('Ichimei', 0.8389753699302673),
    ('Tokyo Sonatı', 0.8376791477203369),
    ('Gizli Kale', 0.8375594615936279),
    ('37 Seconds', 0.8366216421127319),
    ('Ugetsu', 0.8340597748756409),
    ('Geç Sonbahar', 0.8333784341812134),
    ('Cold Fish', 0.8316842913627625),
    ('Cut', 0.8286798596382141),
    ('Chikamatsu monogatari', 0.8266655206680298)]


In [29]:
seed_text = 'Titanik'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Titanik', 0.9382817149162292),
    ('Şöhrete İlk Adım', 0.7013309597969055),
    ('Fake !', 0.6686000227928162),
    ('The Wager', 0.660822868347168),
    ('The Black Hand', 0.6597444415092468),
    ('Boşanma', 0.6544262170791626),
    ('Bu Çocuğun Hayatı', 0.6543473601341248),
    ('Enigma', 0.6496071219444275),
    ('Tutku Oyunları', 0.6491042971611023),
    ('Tatil', 0.6438833475112915),
    ('Hayallerin Peşinde', 0.6434882283210754),
    ('Orlando', 0.640713632106781),
    ('Roosevelt', 0.6402652263641357),
    ('Iris', 0.6394973993301392),
    ("Don's Plum", 0.6316646933555603),
    ('Bu Dünya ve Ötesi', 0.6299759745597839),
    ('Kutsal Duman', 0.6259361505508423),
    ('Okuyucu', 0.6205527782440186),
    ('Amsterdam Ekspres', 0.6186784505844116),
    ('127 Saat', 0.6186205744743347)]


In [30]:
# this dict has title:embedding -> embeddings are created using that specific movie's plot
movie_embeddings = {title: model_dbow.infer_vector(return_token(title)) for title in df_final['Title']}

with open("movie_embed_desc.pickle", "wb") as f:
    pickle.dump(movie_embeddings, f)

#### Model 2

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(row['cleaned']), tags=[row['Title']]) for index, row in df_final.iterrows()]



In [32]:
print(tagged_data[471].tags, ': ', (tagged_data[471].words))


['Yüzüklerin Efendisi: İki Kule'] :  ['yüzüklerin', 'efendisi', 'i̇ki', 'kulede', 'yüzük', 'kardeşliği', 'üyelerinin', 'birinin', 'kardeşlik', 'bozulduktan', 'sonra', 'başlarına', 'gelenler', 'anlatılıyor', 'kahramanlarımız', 'gruplar', 'halinde', 'orta', 'dünya', '’', 'nın', 'tehlikeli', 'yerlerinde', 'maceralar', 'yaşayacaklar', 'yeni', 'kavimler', 'çoktan', 'unutulmuş', 'medeniyetlerle', 'tanışacaklarfrodo', 'sam', 'yanlarında', 'zorunlu', 'işbirliği', 'yapacakları', 'eski', 'bir', 'dost', 'olduğu', 'halde', 'tek', 'yüzük', '’', 'ü', 'düşmanın', 'tam', 'kalbine', 'götürmeye', 'çalışırken', 'diğer', 'hobbitler', 'urukhai', '’', 'nin', 'elinden', 'kurtulabilecek', 'mi', 'karanlık', 'tarafa', 'geçmiş', 'olan', 'saruman', '’', 'ın', 'yaptıkları', 'yanına', 'kalacak', 'gandalf', 'olmadan', 'kahramanlarımızın', 'başarılı', 'olma', 'şansı', 'ne', 'büyük', 'karanlığın', 'gelişi', 'yüzük', 'savaşı', '’', 'na', 'dek', 'olanların', 'anlatılacağı', 'i̇ki', 'kule', 'kuşkusuz', 'üçlemenin', 'heye

In [33]:
model_dbow = Doc2Vec(
    dm=0, dbow_words=1,
    vector_size=200, window=8, epochs=20, workers=12,
)

model_dbow.build_vocab(tagged_data)
print(model_dbow)
    


2023-02-22 18:31:18,282 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow+w,d200,n5,w8,mc5,s0.001,t12>', 'datetime': '2023-02-22T18:31:18.282417', 'gensim': '4.3.0', 'python': '3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-02-22 18:31:18,285 : INFO : collecting all words and their counts
2023-02-22 18:31:18,285 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-02-22 18:31:18,427 : INFO : PROGRESS: at example #10000, processed 823740 words (5833100 words/s), 95690 word types, 9472 tags
2023-02-22 18:31:18,570 : INFO : collected 126855 word types and 17513 unique tags from a corpus of 18492 examples and 1339673 words
2023-02-22 18:31:18,570 : INFO : Creating a fresh vocabulary
2023-02-22 18:31:18,644 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 26434 unique words (20.84% of original 126855, drops 100421)', 'date

Doc2Vec<dbow+w,d200,n5,w8,mc5,s0.001,t12>


In [34]:
model_dbow.train(tagged_data, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)
model_dbow.save("doc2vec.model")


2023-02-22 18:31:18,929 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 12 workers on 26434 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=8 shrink_windows=True', 'datetime': '2023-02-22T18:31:18.929147', 'gensim': '4.3.0', 'python': '3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-02-22 18:31:20,064 : INFO : EPOCH 0 - PROGRESS: at 46.21% examples, 534037 words/s, in_qsize 23, out_qsize 0
2023-02-22 18:31:21,044 : INFO : EPOCH 0: training on 1339673 raw words (1115014 effective words) took 2.1s, 528532 effective words/s
2023-02-22 18:31:22,068 : INFO : EPOCH 1 - PROGRESS: at 36.82% examples, 495621 words/s, in_qsize 23, out_qsize 0
2023-02-22 18:31:23,084 : INFO : EPOCH 1 - PROGRESS: at 87.14% examples, 492978 words/s, in_qsize 14, out_qsize 0
2023-02-22 18:31:23,214 : INFO : EPOCH 1: training on 1339673 raw words (1115483 effective words) took 2

In [35]:
def search_by_tag(tag):
    for doc in tagged_data:
        if doc.tags[0] == tag:
            return " ".join(doc.words)

def return_most_similar(mov_title) -> list:
  tokens = remove_html(remove_punctuation(remove_stop_words(make_lower_case(search_by_tag(mov_title))))).split()
  vector = model_dbow.infer_vector(tokens)
  most_similar = model_dbow.dv.most_similar([vector], topn=20) 
  return most_similar

def return_token(mov_title) -> list:
  tokens = remove_html(remove_punctuation(remove_stop_words(make_lower_case(search_by_tag(mov_title))))).split()
  return tokens

def return_most_similar_token(vec) -> list:
  # len(df_final['Title']
  most_similar = model_dbow.dv.most_similar([vec],topn=20)
  return most_similar
pp = pprint.PrettyPrinter(indent=4)
seed_text = 'Iron Man'
x = return_most_similar(seed_text)
pp.pprint(x)
print(search_by_tag(seed_text))

[   ('Iron Man', 0.9322595000267029),
    ('Superlópez', 0.4410777688026428),
    ("Oflu Hoca Trakya'da", 0.4244818687438965),
    ('Rounding Third', 0.4170389473438263),
    ('Kickboxer: Vengeance', 0.40591973066329956),
    ('Xin Shen Bang: Ne Zha Chongsheng', 0.40542253851890564),
    ('Tron', 0.4036935865879059),
    ('Ölümsüzlerin Savaşı', 0.40248310565948486),
    ('Kontroll', 0.40003448724746704),
    ('Ani Ölüm', 0.39879336953163147),
    ('Operation Seawolf', 0.3970690667629242),
    ('Avengers Confidential: Black Widow & Punisher', 0.39299845695495605),
    ('Hard Kill', 0.3912982940673828),
    ('Hava Kuvvetleri Bir', 0.39031556248664856),
    ('Sessiz Düşman', 0.388791561126709),
    ("Entebbe'de 7 Gün", 0.3879695534706116),
    ('Vexille', 0.38727155327796936),
    ('Akıllı Ol', 0.38670727610588074),
    ('Kayıp Savaş', 0.3865184187889099),
    ('Görünmez Savaşçı', 0.38609176874160767)]
tony stark bir mühendislik dahisi tam bir playboydur kendi ülkesinde teknoloji harikası

In [36]:
seed_text = 'Dune:Çöl Gezegeni'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Dune:Çöl Gezegeni', 0.9543482065200806),
    ('Dune', 0.45587852597236633),
    ('House Of Re-Animator', 0.4497130811214447),
    ('Dune: Part Two', 0.4394340515136719),
    ('Vampir İmparatorluğu', 0.4189107418060303),
    ('Bride Of Frankenstein', 0.4171842932701111),
    ('Cesur Horoz', 0.41203776001930237),
    ('Before We Vanish', 0.39964598417282104),
    ('Cabin Fever', 0.39545121788978577),
    ('Khers nist', 0.3914357125759125),
    ('Avatar 5', 0.38993018865585327),
    ('Xin long men ke zhan', 0.38973021507263184),
    ('Kaptan Harlock', 0.388936311006546),
    ('Resident Evil 5: İntikam', 0.38832470774650574),
    ('Tanrının Kitabi', 0.38437628746032715),
    ('Casuslar Köprüsü', 0.3836266100406647),
    ('The Caine Mutiny Court-Martial', 0.3832657039165497),
    ('28 Gün Sonra', 0.3817640542984009),
    ('Ant-Man ve Wasp: Quantumania', 0.3809562921524048),
    ('Star Wars: Güç Uyanıyor', 0.38010355830192566)]


In [37]:
seed_text = 'Yüzüklerin Efendisi: İki Kule'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Yüzüklerin Efendisi: İki Kule', 0.9500899910926819),
    ('Pearl Harbor', 0.47223082184791565),
    ('Gelibolu', 0.4592815935611725),
    ('King Kong vs. Godzilla', 0.4275374710559845),
    ('The Lord Of The Rings: The War Of Rohirrim', 0.4228450357913971),
    ('Hobbit: Beş Ordunun Savaşı', 0.41768234968185425),
    ('Uzay Yolu IV', 0.4161980152130127),
    ('Murder in the First', 0.4150218665599823),
    ('Harry Potter ve Ateş Kadehi', 0.4138086140155792),
    ('The Big Black', 0.41283881664276123),
    ('Gezgin Pantolon Kardeşliği 2', 0.4121832847595215),
    ('Pers Prensi: Zamanın Kumları', 0.40483254194259644),
    ('The Day It Came to Earth', 0.39698025584220886),
    ('Skinwalkers', 0.3954154849052429),
    ('Da Vinci Şifresi', 0.3941201865673065),
    ('Rescue Dawn', 0.39369285106658936),
    ('Under sandet', 0.3923293650150299),
    ("Mortal Kombat Legends : Scorpion's Revenge", 0.39112597703933716),
    ('Üçkağıtçı', 0.39096465706825256),
    ('Studio 54', 0.39092266559

In [38]:
seed_text = 'Rashômon'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Rashômon', 0.9476543664932251),
    ('Françoise ou la vie conjugale', 0.5435649156570435),
    ('İntikam Meleği/Kadın Hamlet', 0.5181604027748108),
    ('Gölgeler ve Sis', 0.4887164831161499),
    ('Seules Les Bêtes', 0.4784468710422516),
    ('Deathstalker', 0.4713344871997833),
    ('Münferit', 0.4707236588001251),
    ('Ölüm Gemisi', 0.4622574746608734),
    ('Parçalanma', 0.46023598313331604),
    ('Kanlı Feryad', 0.458997517824173),
    ("Fatmagül'ün Suçu Ne?", 0.4543512165546417),
    ('Korkunç Orakçı', 0.45266059041023254),
    ('Aşk Nöbeti', 0.4518894553184509),
    ('Absentia', 0.45114442706108093),
    ('Detained', 0.44956687092781067),
    ('Kehanet; Ayasofya', 0.44840914011001587),
    ('Yılanların Öcü', 0.44721612334251404),
    ('De Lift', 0.44244807958602905),
    ('Matar a un Muerto', 0.442259818315506),
    ('The Exorcist III', 0.44154855608940125)]


In [39]:
seed_text = 'Titanik'
x = return_most_similar(seed_text)
pp.pprint(x)

[   ('Titanik', 0.9488952159881592),
    ('Dinozor', 0.4613831341266632),
    ('Beavis and Butt-Head Do the Universe', 0.4197002351284027),
    ('The Lord Of The Rings: The War Of Rohirrim', 0.4190202057361603),
    ('Last Words', 0.4082288146018982),
    ('Anchiporuno', 0.40757089853286743),
    ('Apollo 10 1/2: Uzay Çağında Çocuk Olmak', 0.3962409496307373),
    ('Nos Futurs', 0.39575523138046265),
    ('İyilik ve Kötülük Okulu', 0.39204472303390503),
    ('Spider-Man: Beyond The Spider-Verse', 0.38597914576530457),
    ('2 Kız', 0.3858177959918976),
    ('Frankenstein: Ölümsüzlerin Savaşı', 0.3837279677391052),
    ('Cirque du Soleil: Worlds Away', 0.38000935316085815),
    ('Shirin', 0.37940600514411926),
    ('Booksmart', 0.37793421745300293),
    ('2012', 0.37791845202445984),
    ('Summer Of 8', 0.3772253394126892),
    ('Vic + Flo ont vu un ours', 0.3758023977279663),
    ('Pas ve Gol', 0.3735332190990448),
    ('Ruh Eşim', 0.3732932507991791)]


In [40]:
# this dict has title:embedding -> embeddings are created using that specific movie's plot
movie_embeddings = {title: model_dbow.infer_vector(return_token(title)) for title in df_final['Title']}

with open("movie_embed_genre.pickle", "wb") as f:
    pickle.dump(movie_embeddings, f)

#### Combining our embeddings and saving them


In [41]:
with open('movie_embed_desc.pickle', 'rb') as f:
    desc = pickle.load(f)
with open('movie_embed_genre.pickle', 'rb') as f_1:
    other = pickle.load(f_1)

In [46]:
def return_rank_list(embeds:dict, mov_title:str) -> list:

  selected_vector = embeds[mov_title]

  similarities = []
  for movie_title, vector in embeds.items():
      cos_sim = 1 - distance.cosine(selected_vector, vector)
      similarities.append((movie_title, cos_sim))

  ranked_movies = sorted(similarities, key=lambda x: x[1], reverse=True)
  ranked_movies = [x for x in ranked_movies if x[0] != mov_title]


  dct = {}
  # replacing cosine similarities with ranks
  for index, item in enumerate(ranked_movies):
    itemlist = list(item)
    # itemlist[1] = index + 1
    # item = tuple(itemlist)

    # ranked_movies[index] = item

    dct[itemlist[0]] = index+1


  return dct

def merge_dicts(dict1, dict2):
    merged_dict = {}
    for title, embedding1 in dict1.items():
        embedding2 = dict2[title]
        merged_embedding = (embedding1 * 0.7 + embedding2 * 0.3)
        merged_dict[title] = merged_embedding
    return merged_dict
merged_dict = merge_dicts(desc, other)


In [ ]:
mov_title = 'Hababam Sınıfı Uyanıyor'
closest_movies = return_rank_list(merged_dict, mov_title)


In [48]:
with open("embeddings/movie_embeddings.pickle", "wb") as f:
    pickle.dump(merged_dict, f)

### Reading from pickle


In [49]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gensim
import nltk
from scipy.spatial import distance
import re
from random import shuffle
import string
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec    
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn import utils
from sklearn.model_selection import train_test_split
import ast
import pickle
import pprint
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
with open('embeddings/movie_embeddings.pickle', 'rb') as f_1:
    embeds = pickle.load(f_1)
    
def return_rank_list(embeds:dict, mov_title:str) -> list:

  selected_vector = embeds[mov_title]

  similarities = []
  for movie_title, vector in embeds.items():
      cos_sim = 1 - distance.cosine(selected_vector, vector)
      similarities.append((movie_title, cos_sim))

  ranked_movies = sorted(similarities, key=lambda x: x[1], reverse=True)
  ranked_movies = [x for x in ranked_movies if x[0] != mov_title]


  dct = {}
  # replacing cosine similarities with ranks
  for index, item in enumerate(ranked_movies):
    itemlist = list(item)
    # itemlist[1] = index + 1
    # item = tuple(itemlist)

    # ranked_movies[index] = item

    dct[itemlist[0]] = index+1


  return dct


In [4]:
# guess_movie = 'Iron Man'
# valid_movies = list(embeds.keys())
# game = True
# sim_movies_list = return_rank_list(embeds,guess_movie)
# while game:
#     user_input = input()
#     if user_input not in valid_movies:
#         print('I do not know this movie')
#     else:
#         if (user_input == guess_movie):
#             print('You have won.')
#             break
#         print(f'{user_input}:{sim_movies_list[user_input]}')



In [ ]:
import customtkinter
import tkinter as tk
guess_movie = "Hababam Sınıfı Uyanıyor"
valid_movies = list(embeds.keys())
sim_movies_list = return_rank_list(embeds,guess_movie)


#### Tkinter

In [8]:
def main_window():
    customtkinter.set_appearance_mode("dark")
    customtkinter.set_default_color_theme("dark-blue")

    root = customtkinter.CTk()
    root.title("Movie Guess Game")
    root.geometry("650x450")
    frame = customtkinter.CTkFrame(master=root)
    frame.pack(pady=15, padx=30, fill="both",expand=True)

    label = customtkinter.CTkLabel(frame, text="Guess the movie", font=("Helvetica", 16))
    label.pack(pady=15, padx=30, fill="both",expand=True)

    guess_entry = customtkinter.CTkEntry(frame, font=("Helvetica", 14))
    guess_entry.pack(pady=10)

    previous_guesses = []
    def game_check(user_input):
        if (user_input == guess_movie):
            result_label.configure(text="You have won. You may close the window.")
            guess_entry.configure(state="disabled")
            guess_button.configure(state="disabled")

    def check_guess():
        user_input = guess_entry.get().strip() 
        game_check(user_input)    
        if user_input not in valid_movies:
            result_label.configure(text="I do not know this movie")
        else:
            game_check(user_input)
            previous_guesses.append((user_input, sim_movies_list[user_input]))
            previous_guesses.sort(key=lambda x: x[1], reverse=False)
            result_label.configure(text=f"{user_input}:{sim_movies_list[user_input]}")
        previous_guesses_list.delete(0, tk.END)
        for guess in previous_guesses:
            previous_guesses_list.insert(tk.END, f"{guess[0]}:{guess[1]}")

    guess_button = customtkinter.CTkButton(frame, text="Guess", command=check_guess, font=("Helvetica", 14))
    guess_button.pack(pady=10)

    result_label = customtkinter.CTkLabel(master=frame, text=f"")
    result_label.pack(pady=20)

    previous_guesses_label = customtkinter.CTkLabel(frame, text="Previous guesses:", font=("Helvetica", 14))
    previous_guesses_label.pack(pady=10)


    previous_guesses_list = tk.Listbox(frame, fg = "white" ,bg = "#262629", font=("Helvetica", 14))
    previous_guesses_list.pack(pady=10)

    root.mainloop()

main_window()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 553, in _clicked
    self._command()
  File "C:\Users\User\AppData\Local\Temp\ipykernel_11340\45447972.py", line 31, in check_guess
    previous_guesses.append((user_input, sim_movies_list[user_input]))
KeyError: 'Recep İvedik 5'


In [6]:
guess_movie = "Recep İvedik 5"
valid_movies = list(embeds.keys())
sim_movies_list = return_rank_list(embeds,guess_movie)
sim_movies_list

{'Recep İvedik 3': 1,
 'Recep İvedik 4': 2,
 'Recep İvedik': 3,
 'Recep İvedik 7': 4,
 'Recep İvedik 2': 5,
 'Osman Pazarlama': 6,
 'Celal ile Ceren': 7,
 'Recep İvedik 6': 8,
 'Kuru Otlar Üstüne': 9,
 'Feride': 10,
 'Yalancı Şahit': 11,
 'Bulutların Üstünde': 12,
 'Bayram Şekeri': 13,
 'Kayhan': 14,
 'Dedemin Fişi': 15,
 'Hep Yek 5: Bizim Şeyimiz mi Altan': 16,
 'Avanak Dedektör': 17,
 'Güvercin Uçuverdi': 18,
 'Hep Yek 2': 19,
 '41 Kere Maşallah': 20,
 'Posta Kutusu': 21,
 "Abbas'ın Melekleri": 22,
 '7 Melek': 23,
 'Hep Yek 4: Bela Okuma Altan': 24,
 'Leblebi Tozu': 25,
 'Öt Bakalım': 26,
 'Ölü Yatırım': 27,
 'Hep Yek 3': 28,
 'Tilki Yuvası': 29,
 'Kaygı': 30,
 'Zengo': 31,
 'Kayseri Aslanı: Çin İşi': 32,
 'Eyvah Karım': 33,
 'Deliha 2': 34,
 'Figüran': 35,
 'Baba Mirası': 36,
 'Çam Yarması 2': 37,
 'Arınma Seansı - Tarot': 38,
 'Sağ Salim': 39,
 'Aşkın Tadı': 40,
 'Muhalif Başkan': 41,
 'Kafkas': 42,
 'Deli Dumrul': 43,
 'Vezir Parmağı': 44,
 'Ulan Salih': 45,
 'Gülcemal': 46,
 'Boğ

In [ ]:
def main_window():
    root = tk.Tk()
    root.title("Movie Guess Game")
    root.geometry("650x450")
    root.config(bg='#303030')

    label = tk.Label(root, text="Guess the movie", bg='#303030', fg='white', font=("Helvetica", 16))
    label.pack(pady=20)

    guess_entry = tk.Entry(root, bg='white', fg='#303030', font=("Helvetica", 14))
    guess_entry.pack(pady=10)

    previous_guesses = []

    def check_guess():
        
        user_input = guess_entry.get()
        if (user_input == guess_movie):
            result_label.config(text="You have won.", bg='#303030', fg='white')
            guess_entry.config(state="disabled")
            guess_button.config(state="disabled")
        if user_input not in valid_movies:
            result_label.config(text="I do not know this movie", bg='#303030', fg='white')
        else:
            previous_guesses.append((user_input, sim_movies_list[user_input]))
            previous_guesses.sort(key=lambda x: x[1], reverse=False)
            if (user_input == guess_movie):
                result_label.config(text="You have won.", bg='#303030', fg='white')
                guess_entry.config(state="disabled")
                guess_button.config(state="disabled")
        previous_guesses_list.delete(0, tk.END)
        for guess in previous_guesses:
            previous_guesses_list.insert(tk.END, f"{guess[0]}:{guess[1]}")

    guess_button = tk.Button(root, text="Guess", command=check_guess, bg='#303030', fg='white', font=("Helvetica", 14))
    guess_button.pack(pady=10)

    result_label = tk.Label(root, bg='#303030', fg='white')
    result_label.pack(pady=20)

    previous_guesses_label = tk.Label(root, text="Previous guesses:", bg='#303030', fg='white', font=("Helvetica", 14))
    previous_guesses_label.pack(pady=10)

    previous_guesses_list = tk.Listbox(root, bg='white', fg='#303030', font=("Helvetica", 14))
    previous_guesses_list.pack(pady=10)

    root.mainloop()

main_window()